# Генерация из GPT

Будем генерировать твиты по примерам из аккаунта "Усы Пескова": https://twitter.com/Sandy_mustache

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 40.0 MB/s 
     |████████████████████████████████| 182 kB 67.6 MB/s 


In [2]:
!wget https://gist.githubusercontent.com/avidale/d3da0ded85a4a16db6eb84d8331638ce/raw/a188084e5ef37b43b01fef0534b55c865b9a569e/tweets.txt

--2022-12-04 13:08:07--  https://gist.githubusercontent.com/avidale/d3da0ded85a4a16db6eb84d8331638ce/raw/a188084e5ef37b43b01fef0534b55c865b9a569e/tweets.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4659 (4.5K) [text/plain]
Saving to: ‘tweets.txt’

tweets.txt          100%[===================>]   4.55K  --.-KB/s    in 0s      

2022-12-04 13:08:07 (56.7 MB/s) - ‘tweets.txt’ saved [4659/4659]



In [6]:
with open('tweets.txt', 'r') as f:
    tweets = f.read().strip().split('\n\n')
print(len(tweets))
for i in range(10, 16):
    print(tweets[i])

26
Знаете, это тот случай, когда Россия может на 400% доверять своему партнеру
Тонер, картриджи и много-много бумаги
Ну это примерно как Пескову поручить расследовать информацию о его часах
Но тут главное, чтобы тракторы и бульдозеры были в порядке - и продолжали качественно давить вредные иностранные продукты
В сеть утекли кадры с репетиции следующей инаугурации Путина
- Скажи что-нибудь сладкое?
- Сахар
- Не. Еще слаще
- Мёд
- ЕЩЕ СЛАЩЕ!!!
- Бюджет!!!
- Вооооот


In [7]:
import torch
device = torch.device("cuda")

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [14]:
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
import random

In [16]:
sep = '\n***\n'
# sep = '\n27479153	Sandy_mustache	2021-02-18 16:44:00	'

prefix = sep.join([''] + random.sample(tweets, k=5) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]
print(prefix)


***
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил
***
Когда работа - улёт
***
Тонер, картриджи и много-много бумаги
***
- Скажи что-нибудь сладкое?
- Сахар
- Не. Еще слаще
- Мёд
- ЕЩЕ СЛАЩЕ!!!
- Бюджет!!!
- Вооооот
***
В сеть утекли кадры с репетиции следующей инаугурации Путина
***



In [17]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+128, 
    repetition_penalty=4.2, 
    temperature=0.7,
    num_beams=3,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Смотрю в окно: снег валит...<s>
Праздник к нам приходит! 

А еще сегодня день рождения у моего друга. И я хочу поздравить его от всей души! Пусть он всегда будет таким же веселым, жизнерадостным и позитивным человеком, каким был до того, как попал в этот мир! Желаю ему всего самого наилучшего во всех его начинаниях и свершениях! 
Желаю тебе здоровья, счастья, любви и благополучия! Чтобы все твои мечты сбывались, а планы воплощались в жизнь! Чтоб ты никогда не чувствовал себя обделенным ни в чем! А главное, чтобы тебя окруж


# Диалоги

Логика та же самая, что и с твитами. Только выбрали разделитель в стиле bash.org

In [19]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [24]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'пока': break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
здарова
привет
ты кто
я друг
чей
мой
ты сам себе друг
нет
а кому
мне
пока


In [26]:
for i in range(10):
    print(respond_to_dialog(['Давай поговорим о домашних животных']))

А давай!
Каких?
О, да! А чем ты занимаешься?
О, да. У тебя есть собака?
О, у меня есть собака!
Каких?
У тебя есть кошка?
Я не люблю собак
О каких?
Каких?
